In [1]:
import torch
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

No GPU available. Training will run on CPU.


In [ ]:
import torch
torch.cuda.is_available()

False

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, log_loss
import matplotlib.pyplot as plt
from flwr_datasets.partitioner import IidPartitioner
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader 

/home/zuy/anaconda3/envs/flower_tutorial/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Source: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

In [2]:
"""Load partition df_train_3 data."""
df = pd.read_csv('/home/zuy/Documents/BCU/ML/CSV/df_train_3.csv')
df.drop("Unnamed: 0", axis=1, inplace=True)
dataset = Dataset.from_pandas(df)
partitioner = IidPartitioner(num_partitions=5)
partitioner.dataset = dataset
dataset = partitioner.load_partition(partition_id=1).to_pandas()

# .values to fix: X has feature names, but LogisticRegression was fitted without feature names
# X = dataset.drop('Class', axis=1).values
# y = dataset['Class'].values
# # Split the on edge data: 80% train, 20% test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [3]:
trainloader, testloader= train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset['Class'])

In [4]:
X_train = trainloader.drop('Class', axis=1)
y_train = trainloader['Class']

skearn 
minmaxscale ()

In [ ]:
X_train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Amount
28536,0.544834,0.921701,-0.060181,-0.823946,0.105318,0.090517,-0.907818,0.301038,-0.810353,0.552154,-0.031500,-0.414985,0.463892,0.138274,0.372314,-0.306193
35730,0.850756,-0.314666,0.584753,-0.707728,0.425283,0.636531,0.812520,1.150059,0.889625,-0.610177,-0.058563,-0.515112,-0.818758,-1.491316,1.387307,2.114823
10522,0.824575,0.843705,0.287426,-0.470351,2.405082,0.525068,0.629222,0.065386,-0.076589,-0.548481,1.451546,-0.963727,-0.004566,0.402805,-0.206822,-0.095616
32937,-0.004663,0.544923,0.077508,-0.045872,0.375620,-0.081098,-0.139800,-0.136544,0.056967,0.290506,-0.185244,-0.282374,-0.821401,-1.072505,0.006372,-0.278636
29177,-0.386347,-0.388697,0.495550,0.493308,-0.460490,0.851869,0.173766,0.423927,0.275991,-0.437238,-0.625745,0.656433,0.317799,0.018283,-0.335834,-0.281280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41411,0.588787,0.888975,-0.144245,-0.624796,0.212572,-0.061745,-0.505496,-0.003170,-0.307216,0.318287,0.344063,0.691793,0.773736,-0.366522,0.570380,-0.293807
39212,-0.123966,0.489708,-0.097760,0.632929,0.812796,-0.760210,0.232086,-0.655078,0.420258,0.585224,0.075915,0.706847,0.882414,-0.334412,-0.141753,-0.306054
40665,-0.074422,-0.242867,0.789619,0.733392,0.576917,-0.182604,-0.536166,0.528554,-0.155711,-0.182666,0.168943,-0.234173,-0.773053,-0.922916,0.463810,-0.216423
11358,-0.621570,0.517404,0.094948,0.103966,0.478090,-0.315753,-0.466010,-0.091485,-0.070143,0.203788,-0.201232,0.171330,-0.394828,-0.862745,-0.032462,-0.181072


In [6]:
X_train.shape

(36455, 16)

In [11]:
X_train_test = torch.from_numpy(np.expand_dims(X_train, axis =1))

In [12]:
X_train_test.shape

torch.Size([36455, 1, 16])

In [15]:
X_train_test2 = DataLoader(X_train, batch_size=2, shuffle=False)

In [18]:
data = X_train_test2.dataset.data 
# X_shape = X_train_test2.dataset.data.shape

AttributeError: 'DataFrame' object has no attribute 'data'

In [17]:
lg = LogisticRegression(max_iter=50, penalty="l2", warm_start=False)
# get initial globlal parameters

n_classes = 2  # Number of class in dataset (y) 
n_features = 16  # Number of features in dataset
lg.classes_ = np.array([i for i in range(n_classes)])

lg.coef_ = np.zeros((n_classes, n_features))
if lg.fit_intercept:
    lg.intercept_ = np.zeros((n_classes,))

In [26]:
# model.coef_ = params[0]
# if model.fit_intercept:
#     model.intercept_ = params[1]

In [18]:
lg_model = lg.fit(X_train, y_train)
fpr, tpr, _ = roc_curve(y_test, lg_model.predict_proba(X_test)[:, 1])
precision, recall, thresholds = precision_recall_curve(y_test, lg_model.predict_proba(X_test)[:, 1])
# plot_roc(fpr, tpr)
# plot_prc(precision, recall)
print(classification_report(y_test, lg_model.predict(X_test), target_names=['Not Fraud', 'Fraud']))
print("ROC_AUC:", roc_auc_score(y_test, lg_model.predict_proba(X_test)[:, 1]))
print("AUC:", auc(recall, precision))
print("Log_loss:", log_loss(y_test,lg_model.predict_proba(X_test)))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00      9102
       Fraud       0.75      0.75      0.75        12

    accuracy                           1.00      9114
   macro avg       0.87      0.87      0.87      9114
weighted avg       1.00      1.00      1.00      9114

ROC_AUC: 0.9997619570790303
AUC: 0.8505723905723906
Log_loss: 0.0020138044875679696


In [6]:
rnn = nn.RNN(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, h1 = rnn(input, h0)

In [9]:
output2, h2 = rnn(input, h1)